In [2]:
import numpy as np
import cv2
import time

import tkinter as tk
from PIL import Image, ImageTk

In [3]:
def calculate_disparity_matrix(imgL, imgR,min_disp, max_disp, disp_size, window_radius):
    num_disp = max_disp-min_disp
    h, w = imgL.shape
    
    #The cropped center area we are interested in
    croph = [(h-disp_size)//2,(h+disp_size)//2]
    cropw = [(w-disp_size)//2,(w+disp_size)//2]

    

    ref_img_cropped = imgL[croph[0]:croph[1],cropw[0]:cropw[1]]

    #Simple sum kernel for our filter
    kernel = np.ones([window_radius, window_radius])

    #C(y,x,d) is the 3D volume containing the cost of a disparity at pixel x,y in the reference
    C = np.zeros(
        [disp_size, disp_size, max_disp])

    for d in range(min_disp, max_disp):
        # Shift image d pixels horizontally
        translation_matrix = np.float32([[1, 0, d], [0, 1, 0]])
        shifted_image = cv2.warpAffine(
            imgR, translation_matrix,
            (imgR.shape[1], imgR.shape[0]))
        shifted_image_cropped = shifted_image[croph[0]:croph[1],cropw[0]:cropw[1]]
        # Absolute difference
        AD = abs(np.float32(ref_img_cropped) - np.float32(shifted_image_cropped))
        # Calculate SAD.
        C[:, :, d] = cv2.filter2D(AD, -1, kernel)
    #Add min_disp as the index is shifted
    disparity = min_disp + np.argmin(C[:,:,min_disp:max_disp], axis=2)
    return disparity

In [4]:
def get_distance_from_disparity(main_disp, f, b):
    distance = (f*b)/(main_disp*1000)
    return distance

In [5]:
def calculate_disparity_matrix_dynamic_range(imgL, imgR,min_disp, max_disp, disp_size, window_radius, main_disp):
    num_disp = max_disp-min_disp
    h, w = imgL.shape
    
    #Use the previous main_disp to offset disparity search around the main disparity
    disp_offset = int(main_disp - max_disp//2)
    if(disp_offset>0):
        max_disp += disp_offset
        min_disp += disp_offset
    print(disp_offset)

    #The cropped center area we are interested in
    croph = [(h-disp_size)//2,(h+disp_size)//2]
    cropw = [(w-disp_size)//2,(w+disp_size)//2]

    ref_img_cropped = imgL[croph[0]:croph[1],cropw[0]:cropw[1]]

    #Simple sum kernel for our filter
    kernel = np.ones([window_radius, window_radius])

    #C(y,x,d) is the 3D volume containing the cost of a disparity at pixel x,y in the reference
    C = np.zeros(
        [disp_size, disp_size, max_disp])

    for d in range(min_disp, max_disp):
        # Shift image d pixels horizontally
        translation_matrix = np.float32([[1, 0, d], [0, 1, 0]])
        shifted_image = cv2.warpAffine(
            imgR, translation_matrix,
            (imgR.shape[1], imgR.shape[0]))
        shifted_image_cropped = shifted_image[croph[0]:croph[1],cropw[0]:cropw[1]]
        # Absolute difference
        AD = abs(np.float32(ref_img_cropped) - np.float32(shifted_image_cropped))
        # Calculate SAD.
        C[:, :, d] = cv2.filter2D(AD, -1, kernel)
    #Add min_disp as the index is shifted
    disparity = min_disp + np.argmin(C[:,:,min_disp:max_disp], axis=2)
    return disparity

In [8]:
#Parameters

#Stereo cameras
focal_length = 567.2 #pixels
baseline = 92.226 #mm

#Stereo matching algorithm
min_disp=0
max_disp=128

num_disp = max_disp-min_disp
disparity_area_size = 100
window_radius = 8

In [22]:
vidL = cv2.VideoCapture('../data/robotL.avi')
vidR = cv2.VideoCapture('../data/robotR.avi')
if (vidL.isOpened()==False or vidR.isOpened()==False):
  print("Error opening video")

# Read until video is completed
while(vidL.isOpened()):
      # Capture frame-by-frame
      retL, frameL = vidL.read()
      retR, frameR = vidR.read()
      if retL == True and retR == True:
        frameL_greyscale=cv2.cvtColor(frameL, cv2.COLOR_BGR2GRAY)
        frameR_greyscale=cv2.cvtColor(frameR, cv2.COLOR_BGR2GRAY)
        
    
        start = time.time()
        
        disparity = calculate_disparity_matrix(frameL_greyscale, frameR_greyscale,min_disp, max_disp, disparity_area_size, window_radius)
        main_disp = np.mean(disparity)
        
        distance = get_distance_from_disparity(main_disp, focal_length, baseline)
        print("FPS: ", 1/(time.time()-start))
        print("Distance to obstacle(m): ", distance)
		    
        # Scale disparity values for visualization 
        disparity_visual = np.uint8(disparity * 255 / num_disp)
          
        # Show frames
        cv2.imshow('Image', frameL)
        cv2.imshow('Disparity', disparity_visual)
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

      # Break the loop
      else: 
          break

# When everything done, release the video capture object
vidL.release()
vidR.release()
# Closes all the frames
cv2.destroyAllWindows()


FPS:  8.171252678745374
Distance to obstacle(m):  2.6475514953360904
FPS:  7.847653545782145
Distance to obstacle(m):  2.6329728048320122
FPS:  8.768714224190196
Distance to obstacle(m):  2.616039487700102
FPS:  6.894512076849992
Distance to obstacle(m):  2.656411533500574
FPS:  8.7131737211114
Distance to obstacle(m):  2.603669644768081
FPS:  8.823704888028695
Distance to obstacle(m):  2.651160963347389
FPS:  7.368912610749681
Distance to obstacle(m):  2.653500958719273
FPS:  8.961460394241493
Distance to obstacle(m):  2.6438983897217145
FPS:  8.922171712766886
Distance to obstacle(m):  2.6250583217077996
FPS:  8.770914626750814
Distance to obstacle(m):  2.571441987130645
FPS:  8.550189480808315
Distance to obstacle(m):  2.641415229246617
FPS:  6.70566134867711
Distance to obstacle(m):  2.6570996845650194
FPS:  8.855748746371074
Distance to obstacle(m):  2.632416310632709
FPS:  8.189073046157331
Distance to obstacle(m):  2.608590509245407
FPS:  6.47679932796882
Distance to obstacle(m)

In [13]:
class MainWindow():
    def __init__(self, window):
        self.use_dynamic_disparity_range = False
        self.calculate_checkerboard = True
        self.main_disp = max_disp//2

        self.window = window
        self.vidL = cv2.VideoCapture('../data/robotL.avi')
        self.vidR = cv2.VideoCapture('../data/robotR.avi')
        if (self.vidL.isOpened()==False or self.vidR.isOpened()==False):
            print("Error opening video")

        self.width = 800
        self.height = 600
        self.interval = 1 # Interval in ms to get the latest frame

        self.distance_widget = tk.Label(root, text="Distance: 0", width=100)
        self.distance_widget.config(font=("Courier", 20))
        self.fps_widget = tk.Label(root, text="FPS: 0", width=100)
        self.fps_widget.config(font=("Courier", 20))

        # Create canvas for gui
        self.canvas = tk.Canvas(self.window, width=self.width, height=self.height)
        self.canvas.grid(row=0, column=0)
        # Update gui
        self.update_gui()
    def update_gui(self):

        # Capture frame-by-frame
        retL, frameL = self.vidL.read()
        retR, frameR = self.vidR.read()
        if retL == True and retR == True:
            frameL_greyscale=cv2.cvtColor(frameL, cv2.COLOR_BGR2GRAY)
            frameR_greyscale=cv2.cvtColor(frameR, cv2.COLOR_BGR2GRAY)
            
        
            start = time.time()
            
            if(self.use_dynamic_disparity_range):
                disparity = calculate_disparity_matrix_dynamic_range(frameL_greyscale, frameR_greyscale,min_disp, max_disp, 
                disparity_area_size, window_radius, self.main_disp)
            else:
                disparity = calculate_disparity_matrix(frameL_greyscale, frameR_greyscale,min_disp, max_disp, 
                disparity_area_size, window_radius)
            
            self.main_disp = np.mean(disparity)
            
            distance = get_distance_from_disparity(self.main_disp, focal_length, baseline)
            fps = 1/(time.time()-start)
            #print("FPS: ", 1/(time.time()-start))
                
            # Scale disparity values for visualization 
            disparity_visual = np.uint8(disparity * 128 / num_disp)
            disp_heatmap = cv2.applyColorMap(disparity_visual, cv2.COLORMAP_JET)
    
            blue,green,red = cv2.split(frameL)
            self.image_rgb = cv2.merge((red,green,blue))
            self.image_rgb = Image.fromarray(self.image_rgb)
            self.image_rgb = ImageTk.PhotoImage(image=self.image_rgb, master=root)

           # Update image
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.image_rgb)

            blue,green,red = cv2.split(disp_heatmap)
            self.image_disp = cv2.merge((red,green,blue))
            self.image_disp = Image.fromarray(self.image_disp)
            self.image_disp = ImageTk.PhotoImage(image=self.image_disp, master=root)
            
            self.canvas.create_image(680, 250, anchor=tk.NW, image=self.image_disp)
            if(distance<0.8):
                self.distance_widget["text"] = "ALARM! Distance to obstacle: "+str(round(distance,2)) + "m"
                self.distance_widget.config(fg="red")
            else:
                self.distance_widget["text"] = "Distance to obstacle: "+str(round(distance,2)) + "m"
                self.distance_widget.config(fg="black")
            self.distance_widget.grid()

            self.fps_widget["text"] = "FPS: " + str(round(fps))
            self.fps_widget.grid()
            # Repeat every 'interval' ms
            self.window.after(self.interval, self.update_gui)
            

        # Break the loop
        else:             
            # When everything done, release the video capture object
            vidL.release()
            vidR.release()
            # Closes all the frames
            cv2.destroyAllWindows()
            quit()
if __name__ == "__main__":
    root = tk.Tk()
    MainWindow(root)
    root.mainloop()